In [1]:
from numpy import array
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.utils import to_categorical
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD
import tensorflow as tf

from sklearn.metrics import r2_score
import pandas as pd
# from sodapy import Socrata



In [2]:
#Uncomment the below to open data source, download sensor data, and store it as a csv locally.

##Function to get Sensor count history data
# def sensor_count():
#     client = Socrata('data.melbourne.vic.gov.au', 'nlPM0PQJSjzCsbVqntjPvjB1f', None)
#     sensor_data_id = "b2ak-trbp"
#     results = client.get(sensor_data_id, limit=5000000)
#     df = pd.DataFrame.from_records(results)
#     df = df[['date_time', 'year', 'month', 'mdate', 'day', 'time', 'sensor_id', 'sensor_name', 'hourly_counts']]
#     return df

# sensor_history = sensor_count()

# sensor_history.to_csv('sensor_history.csv', index=False)

In [31]:
sensor_history = pd.read_csv('sensor_history.csv')

In [32]:
single_sensor = sensor_history[sensor_history.sensor_id == 4].copy()
single_sensor.sort_values('date_time', inplace=True, ascending=True)

In [33]:
#Make sensor counts range from 0 to 1.
maximum = single_sensor.hourly_counts.max()
single_sensor['hourly_counts'] = single_sensor['hourly_counts']/maximum

In [34]:
sensor_seq = []
for i in range(0, len(single_sensor)):
    sensor_seq.append(single_sensor.hourly_counts.iloc[i])

In [75]:
# split a univariate sequence
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# choose a number of time steps
n_steps = 196
# split into samples
X, y = split_sequence(sensor_seq, n_steps)


In [76]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))


In [77]:
X_train = X[10001:80000]
X_val = X[80001:]
X_test = X[:10000]
Y_train = y[10001:80000]
Y_val = y[80001:]
Y_test = y[:10000]

In [80]:
epochs = 50
learning_rate = 0.00001
decay_rate = learning_rate / epochs
momentum = 0.8

# define model
model = Sequential()
# model.add(LSTM(168, activation='relu', input_shape=(n_steps, n_features)))
# model.add(Dropout(0.2))
# model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
# model.add(LSTM(24, return_sequences=True, input_shape=(n_steps, n_features)))
# model.add(Dropout(0.1))
model.add(LSTM(48, return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.15))
model.add(LSTM(96, return_sequences=True, input_shape=(n_steps, n_features)))
model.add(Dropout(0.15))
model.add(LSTM(120, input_shape=(n_steps, n_features)))
model.add(Dropout(0.15))
model.add(Dense(48, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(optimizer=opt, loss='mse')


In [81]:
# fit model
model.fit(X_train, Y_train, validation_data =(X_val, Y_val), epochs=45, verbose=1, batch_size = 64)


Epoch 1/45
1094/1094 [==============================] - 562s 514ms/step - loss: 0.0532 - val_loss: 0.0214
Epoch 2/45
1094/1094 [==============================] - 560s 512ms/step - loss: 0.0258 - val_loss: 0.0223
Epoch 3/45
1094/1094 [==============================] - 563s 515ms/step - loss: 0.0256 - val_loss: 0.0238
Epoch 4/45
1094/1094 [==============================] - 569s 520ms/step - loss: 0.0254 - val_loss: 0.0262
Epoch 5/45
1094/1094 [==============================] - 568s 519ms/step - loss: 0.0162 - val_loss: 0.0060
Epoch 6/45
1094/1094 [==============================] - 568s 519ms/step - loss: 0.0045 - val_loss: 0.0039
Epoch 7/45
1094/1094 [==============================] - 562s 513ms/step - loss: 0.0041 - val_loss: 0.0033
Epoch 8/45
1094/1094 [==============================] - 560s 512ms/step - loss: 0.0039 - val_loss: 0.0032
Epoch 9/45
1094/1094 [==============================] - 562s 514ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 10/45
1094/1094 [=======================

In [84]:
# demonstrate prediction
x_input = array(sensor_seq[10001:10197])
# x_input = x_input/maximum
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print("predicted next value: ", yhat * maximum)
print("actual next value: ", sensor_seq[10197] * maximum)

predicted next value:  [[782.87524]]
actual next value:  636.0


In [85]:
# demonstrate prediction
x_input = array(sensor_seq[10021:10217])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print("predicted next value: ", yhat * maximum)
print("actual next value: ", sensor_seq[10217] * maximum)

predicted next value:  [[2895.828]]
actual next value:  3111.0


In [82]:
pred = model.predict(X_test)

In [83]:
r2_score(Y_test, pred)

0.9189010610843554

In [86]:
# model.save("model_2.h5")